In [1]:
import requests

In [2]:
response = requests.get("https://www.topuniversities.com/university-rankings/world-university-rankings/2022")
response.status_code

200

In [3]:
response.content[:2000]

b'\n<!DOCTYPE html>\n<html lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">\n<head attr=\'https://www.topuniversities.com/\'>\n<script>\n    function setCookieTUTM(cname, cvalue, exdays) {\n      const d = new Date();\n      d.setTime(d.getTime() + (exdays*24*60*60*1000));\n      let expires = "expires="+ d.toUTCString();\n      document.cookie = cname + "=" + cvalue + ";" + expires + ";path=/";\n    }\n    function getCookieTUTM(cname) {\n      let name = cname + "=";\n      let decodedCookie = decodeURIComponent(document.cookie);\n      let ca = decodedCookie.split(\';\');\n      for(let i = 0; i <ca.length; i++) {\n        let c = ca[i];\n     

In [4]:
import bs4
soup = bs4.BeautifulSoup(response.content)

In [5]:
data_div = soup.find("div", id='ranking-fillters-div')

In [6]:
data_div.find_all("div", class_="_qs-ranking-data-row")

[]

In [7]:
with open("qs_rankings.html", "wb") as file:
    file.write(response.content)

In [8]:
from selenium import webdriver
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

'/home/martin/mambaforge/envs/pp4rs/lib/python3.10/site-packages/chromedriver_autoinstaller/103/chromedriver'

In [9]:
import time

In [10]:
browser = webdriver.Chrome()
browser.set_window_size(1600, 900)

In [11]:
type(browser)

selenium.webdriver.chrome.webdriver.WebDriver

In [12]:
browser.get("https://www.topuniversities.com/university-rankings/world-university-rankings/2022")
time.sleep(5)


In [13]:
soup = bs4.BeautifulSoup(browser.page_source)

In [14]:
len(soup.find_all("div", class_="_qs-ranking-data-row"))

10

In [15]:
rows = soup.find_all("div", class_="_qs-ranking-data-row")
len(rows)

10

In [16]:
import pandas as pd

def parse_page(soup):

    rows = soup.find_all("div", class_="_qs-ranking-data-row")

    data = []

    for row in rows:
        rank = row.find("div", class_="_univ-rank")
        name = row.find("a", class_="uni-link")
        score = row.find("span", class_="overall-score-span")

        row_data = {
            "rank": rank.text,
            "name": name.text,
            "score": score.text
        }

        data.append(row_data)

    return pd.DataFrame(data)

In [17]:
parse_page(soup)

,rank,name,score
0,1,Massachusetts Institute of Technology (MIT),100
1,2,University of Oxford,99.5
2,=3,Stanford University,98.7
3,=3,University of Cambridge,98.7
4,5,Harvard University,98
5,6,California Institute of Technology (Caltech),97.4
6,7,Imperial College London,97.3
7,=8,ETH Zurich,95.4
8,=8,UCL,95.4
9,10,University of Chicago,94.5


In [18]:
browser = webdriver.Chrome()
browser.set_window_size(1600, 900)
browser.get("https://www.topuniversities.com/university-rankings/world-university-rankings/2022")
time.sleep(5)

In [19]:
from selenium.webdriver.common.by import By
gdpr_button = browser.find_element(By.CSS_SELECTOR, "button.agree-button.eu-cookie-compliance-default-button")
gdpr_button.click()
time.sleep(1)

In [20]:
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

try:
    clear_button = browser.find_element(By.CSS_SELECTOR, "div.no-ranking-results-found") \
        .find_element(By.TAG_NAME, "a")
    clear_button.click()
    time.sleep(1)
except (NoSuchElementException, ElementNotInteractableException):
    pass

In [21]:
next_button = browser.find_element(By.CSS_SELECTOR, "a.page-link.next")
next_button.click()